# 数理最適化
## 線形計画法

$
\begin{align}
Maximize\ &3x + 4y \\
Subject\ to \ &x+4y \le 1700 \\
&2x+3y \le 1400 \\
&2x+y \le 1000 \\
&x \ge 0 \\
&y \ge 0 \\
\end{align}
$

### 線形計画法の一般型

$
\begin{align}
Minimize\ &{\bf c}^{\mathsf{T}}{\bf x} \\
Subject\ to\ &{\bf Gx} \le {\bf h} \\
&{\bf Ax} = {\bf b} \\
\end{align}
$

これに当てはめると

$
\begin{align}
Minmize\  &\left(
        \begin{array}{c}
            -3 \\
            -4
        \end{array}
    \right)
    ^{\mathsf T}
    \left(
        \begin{array}{c}
            x \\
            y
        \end{array}
    \right) \\
Subject\ to\ &\left(
    \begin{array}{cc}
        1 & 4 \\
        2 & 3 \\
        2 & 1 \\
        -1 & 0 \\
        0 & -1
    \end{array}
    \right)
    \left(
        \begin{array}{c}
            x \\
            y
        \end{array}
    \right)
    \le
    \left(
        \begin{array}{c}
            1700 \\
            1400 \\
            1000 \\
            0 \\
            0
        \end{array}
    \right)
\end{align}
$

つまり

$
\begin{align}
{\bf c} =
    \left(
    \begin{array}{c}
        -3 \\
        -4
    \end{array}
    \right),\ 
{\bf G} =
    \left(
        \begin{array}{cc}
            1 & 4 \\
            2 & 3 \\
            2 & 1 \\
            -1 & 0 \\
            0 & -1
        \end{array}
    \right),\ 
{\bf h} =
   \left(
       \begin{array}{c}
           1700 \\
           1400 \\
           1000 \\
           0 \\
           0\\
       \end{array}
   \right)
\end{align}
$

In [6]:
import numpy as np
from scipy import optimize

c = np.array([-3, -4], dtype=np.float64)
G = np.array([[1, 4], [2, 3], [2, 1], [-1, 0], [0, -1]], dtype=np.float64)
h = np.array([1700, 1400, 1000, 0, 0], np.float64)
sol = optimize.linprog(c, A_ub=G, b_ub=h, bounds=(0, None))

print(sol.x)
print(sol.fun)

[399.99988463 199.99996114]
-1999.9994984688606


## 2次計画法

以下の様な2変数の2次関数の最小値を求めるような問題を2次計画問題と呼ぶ。
$$
\begin{align}
f(x, y) = x^2 + xy + y^2 + 2x + 4y
\end{align}
$$

一般の2次計画問題は以下の様に書くことが出来る。
$$
\begin{align}
Minimize\ \ &\frac{1}{2}{\bf x^{\mathsf T}Px + q^{\mathsf T}x} \\
Subject\ to\ \ &{\bf Ax = b} \\
&{\bf Gx \le h}
\end{align}
$$

`cvxopt`モジュールを用いて2次計画問題を解くことを考える。  
`cvxopt`では制約条件なしの2次計画問題の標準形として以下の形を仮定している

$$
\begin{align}
\frac{1}{2}{\bf x^{\top}Px}
\end{align}
$$

$$
\begin{align}
    f(x, y) = \frac{1}{2} 
    \begin{pmatrix} x & y \end{pmatrix}
    \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}
    \begin{pmatrix} x \\ y \end{pmatrix}
    + 
    \begin{pmatrix} 2 & 4 \end{pmatrix}
    \begin{pmatrix} x \\ y \end{pmatrix}
\end{align} \\
\therefore
{\bf P} = \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix},
{\bf q} = \begin{pmatrix} 2 \\ 4 \end{pmatrix}
$$


In [5]:
import numpy as np
import cvxopt

P = cvxopt.matrix(np.array([[2, 1], [1, 2]], dtype=np.float64))
q = cvxopt.matrix(np.array([2, 4], dtype=np.float64))

sol = cvxopt.solvers.qp(P, q)

#print(sol)
print(np.array(sol["x"]))
print(np.array(sol["primal objective"]))

[[ 2.22044605e-16]
 [-2.00000000e+00]]
-4.0


## 制約条件付き問題

`cvxopt`に制約条件として入力出来るのは次のような式

$$
{\bf Ax = b,\ Gx \le h}
$$

### 制約条件が等号式の場合
$$
{\bf Ax = b}
$$

$$
\begin{align}
&Minimize\ f(x, y) = x^2 + xy + y^2 +2x + 4y \\
&Subject\ to\ x + y = 0 \\ 
\end{align}
$$

In [6]:
import numpy as np
import cvxopt

P = cvxopt.matrix(np.array([[2, 1], [1, 2]], dtype=np.float64))
q = cvxopt.matrix(np.array([2, 4], dtype=np.float64))
A = cvxopt.matrix(np.array([[1, 1]], dtype=np.float64))
b = cvxopt.matrix(np.array([0], dtype=np.float64))

sol = cvxopt.solvers.qp(P, q, A=A, b=b)

print(np.array(sol["x"]))
print(np.array(sol["primal objective"]))

[[ 1.]
 [-1.]]
-1.0000000000000016


ここで、$x+y=0$という条件は
$$
\begin{pmatrix} 1 & 1 \end{pmatrix}
\begin{pmatrix} x \\ y \end{pmatrix}
= 0
$$
と同値なので、
$$
{\bf A} = \begin{pmatrix} 1 & 1 \end{pmatrix},\ 
{\bf b} = \begin{pmatrix} 0 \end{pmatrix}
$$
としている

### 制約条件が不等号式の場合
$$
\begin{align}
&{\bf Gx \le h}
\end{align}
$$

$$
\begin{align}
&Minimize\ f(x, y) = x^2 + xy + y^2 + 2x + 4y \\
&Subject\ to\ 2x + 3y \le 3
\end{align}
$$

In [7]:
import numpy as np
import cvxopt

P = cvxopt.matrix(np.array([[2, 1], [1, 2]], dtype=np.float64))
q = cvxopt.matrix(np.array([2, 4], dtype=np.float64))
G = cvxopt.matrix(np.array([[2, 3]], dtype=np.float64))
h = cvxopt.matrix(np.array([3], dtype=np.float64))

sol = cvxopt.solvers.qp(P, q, G=G, h=h)

print(np.array(sol["x"]))
print(np.array(sol["primal objective"]))

     pcost       dcost       gap    pres   dres
 0:  1.8858e+00  2.9758e-01  2e+00  5e-18  2e+00
 1: -2.1066e+00 -2.1546e+00  5e-02  2e-16  7e-01
 2: -3.9999e+00 -4.0665e+00  7e-02  3e-16  2e-16
 3: -4.0000e+00 -4.0007e+00  7e-04  1e-15  1e-16
 4: -4.0000e+00 -4.0000e+00  7e-06  3e-16  6e-17
 5: -4.0000e+00 -4.0000e+00  7e-08  9e-16  2e-16
Optimal solution found.
[[-2.45940172e-09]
 [-2.00000001e+00]]
-4.0


$2x + 3y \le 3$という式は
$$
\begin{pmatrix} 2 & 3 \end{pmatrix}
\begin{pmatrix} x \\ y \end{pmatrix}
\le 3
$$
で表せるので
$$
{\bf G} = \begin{pmatrix} 2 & 3 \end{pmatrix},\ 
{\bf h} = \begin{pmatrix} 3 \end{pmatrix}
$$
と考えられる。